In [1]:
import numpy as np
import cv2
import datetime
import pandas as pd
import keras
import keras.backend as k
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D
from keras.layers import SpatialDropout2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from keras.models import Sequential,load_model
from tensorflow.keras.optimizers import Adam

# from keras.preprocessing import image

In [2]:
mdl=Sequential()
mdl.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
mdl.add(MaxPooling2D() )
mdl.add(Conv2D(32,(3,3),activation='relu'))
mdl.add(MaxPooling2D() )
mdl.add(Conv2D(32,(3,3),activation='relu'))
mdl.add(MaxPooling2D() )
mdl.add(Flatten())
mdl.add(Dense(100,activation='relu'))
mdl.add(Dense(1,activation='sigmoid'))

mdl.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

td = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


mdl.save('mymodel.h5',mdl.fit_generator(
td.flow_from_directory('train',target_size=(150,150),batch_size=16 ,class_mode='binary'),epochs=10,
        validation_data=ImageDataGenerator(rescale=1./255).flow_from_directory('test',target_size=(150,150),batch_size=32,class_mode='binary')))
#To test for individual images

Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


<ipython-input-2-f8e6489d3e92>:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  mdl.save('mymodel.h5',mdl.fit_generator(


Epoch 1/10
83/83 [==============================] - 85s 998ms/step - loss: 0.3459 - accuracy: 0.8463 - val_loss: 0.2233 - val_accuracy: 0.9175
Epoch 2/10
83/83 [==============================] - 52s 629ms/step - loss: 0.1716 - accuracy: 0.9346 - val_loss: 0.3555 - val_accuracy: 0.8918
Epoch 3/10
83/83 [==============================] - 51s 610ms/step - loss: 0.1325 - accuracy: 0.9536 - val_loss: 0.1515 - val_accuracy: 0.9536
Epoch 4/10
83/83 [==============================] - 51s 619ms/step - loss: 0.1122 - accuracy: 0.9559 - val_loss: 0.0872 - val_accuracy: 0.9742
Epoch 5/10
83/83 [==============================] - 51s 614ms/step - loss: 0.0991 - accuracy: 0.9642 - val_loss: 0.1830 - val_accuracy: 0.9330
Epoch 6/10
83/83 [==============================] - 51s 610ms/step - loss: 0.0885 - accuracy: 0.9703 - val_loss: 0.0899 - val_accuracy: 0.9742
Epoch 7/10
83/83 [==============================] - 52s 625ms/step - loss: 0.0902 - accuracy: 0.9650 - val_loss: 0.1034 - val_accuracy: 0.9639

In [ ]:
from keras.models import Sequential,load_model

from keras.preprocessing import image
mm=load_model('mymodel.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    for(x,y,w,h) in face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4):
        cv2.imwrite('temp.jpg',img[y:y+h, x:x+w])
        ti=image.load_img('temp.jpg',target_size=(150,150,3))
        ti=image.img_to_array(ti)
        ti=np.expand_dims(ti,axis=0)
        pred=mm.predict(ti)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('LIVE',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()